<a href="https://colab.research.google.com/github/Malwina9912/Uczenie-maszynowe-/blob/main/Uczenie_maszynowe_w_finansach_case_1_Malwina_Pawlicka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [9]:
# Wczytanie danych
file_path = "/content/case1 (1).csv"
# Określenie separatora (;)
df = pd.read_csv(file_path, sep=';')

In [10]:
# Sprawdzenie braków danych
missing_values = df.isnull().sum()
print("Braki danych:\n", missing_values)


Braki danych:
 Unnamed: 0                                                0
Index                                                     0
Address                                                   0
FLAG                                                      0
Avg min between sent tnx                                  0
Avg min between received tnx                              0
Time Diff between first and last (Mins)                   0
Sent tnx                                                  0
Received Tnx                                              0
Number of Created Contracts                               0
Unique Received From Addresses                            0
Unique Sent To Addresses                                  0
min value received                                        0
max value received                                        0
avg val received                                          0
min val sent                                              0
max val sent             

In [11]:
# Usunięcie braków danych
df.dropna(inplace=True)

In [12]:
# Podział na cechy liczbowe i kategoryczne
numerical_features = df.select_dtypes(include=['float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns

In [13]:
# Pipeline dla cech numerycznych
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [14]:
# Pipeline dla cech kategorycznych
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [15]:
# Połączenie transformacji
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [20]:

# Przygotowanie danych
X = df.drop(columns=['FLAG'])
y = df['FLAG']


In [21]:
# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Przetwarzanie danych
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [23]:
print("Kształt X_train:", X_train_processed.shape)
print("Kształt X_test:", X_test_processed.shape)
print("Kształt y_train:", y_train.shape)
print("Kształt y_test:", y_test.shape)

Kształt X_train: (7184, 7869)
Kształt X_test: (1797, 7869)
Kształt y_train: (7184,)
Kształt y_test: (1797,)


In [32]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

In [33]:

# Standaryzacja danych
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train_processed)
X_test_scaled = scaler.transform(X_test_processed)

In [34]:
# Definicja modelu TruncatedSVD
svd = TruncatedSVD(n_components=10)

In [35]:

# Dopasowanie modelu do danych treningowych
X_train_svd = svd.fit_transform(X_train_scaled)

In [36]:
# Zastosowanie modelu do danych testowych
X_test_svd = svd.transform(X_test_scaled)

In [37]:
# Informacje o wyjaśnionej wariancji
explained_variance = svd.explained_variance_ratio_
print("Wyjaśniona wariancja przez każdą składową:\n", explained_variance)

Wyjaśniona wariancja przez każdą składową:
 [0.00072251 0.00075973 0.00072428 0.00060633 0.00035568 0.00043097
 0.00041277 0.00036974 0.000385   0.00037677]


In [38]:
from imblearn.under_sampling import RandomUnderSampler

In [39]:
# Zbilansowanie częściowej próbki
under_sampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_train_resampled, y_train_resampled = under_sampler.fit_resample(X_train_scaled, y_train)

In [40]:

# Sprawdzenie rozkładu klas po zbilansowaniu
print("Rozkład klas po zbilansowaniu:")
print(pd.Series(y_train_resampled).value_counts())

Rozkład klas po zbilansowaniu:
0    2160
1    1080
Name: FLAG, dtype: int64


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

In [42]:
# Wybór trzech różnych modeli klasyfikacji
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

In [43]:
# Trenowanie modeli
for name, model in models.items():
    model.fit(X_train_resampled, y_train_resampled)

In [44]:
# Ocena modeli
for name, model in models.items():
    print(f"Model: {name}")
    y_pred = model.predict(X_test_scaled)
    print(classification_report(y_test, y_pred))

Model: Random Forest
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1527
           1       0.99      0.92      0.95       270

    accuracy                           0.99      1797
   macro avg       0.99      0.96      0.97      1797
weighted avg       0.99      0.99      0.99      1797

Model: SVM
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1527
           1       1.00      0.80      0.89       270

    accuracy                           0.97      1797
   macro avg       0.98      0.90      0.94      1797
weighted avg       0.97      0.97      0.97      1797

Model: Gradient Boosting
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1527
           1       0.98      0.93      0.95       270

    accuracy                           0.99      1797
   macro avg       0.98      0.96      0.97      1797
weighted avg     

In [ ]:
#
#Analizując wyniki oceny modeli dla każdej klasyfikacji, możemy zauważyć, że dla modelu Random Forest uzyskano następujące wyniki:

#Precyzja (precision) dla klasy 0: 0.99

#Czułość (recall) dla klasy 0: 1.00

#F1-score dla klasy 0: 0.99

#Wsparcie (support) dla klasy 0: 1527

#Precyzja (precision) dla klasy 1: 0.99

#Czułość (recall) dla klasy 1: 0.92

#F1-score dla klasy 1: 0.95

#Wsparcie (support) dla klasy 1: 270

#Analiza tych wyników wskazuje, że model Random Forest jest bardzo skuteczny w klasyfikacji obserwacji klasy 0 (brak oszustw), osiągając wysoką precyzję, czułość i F1-score, co sugeruje, że model minimalizuje liczbę fałszywych alarmów i pominiętych oszustw dla tej klasy. Jednocześnie, dla klasy 1 (obserwacje dotyczące oszustw), model osiąga wysoką precyzję, co oznacza, że większość przewidywanych przez model przypadków oszustw rzeczywiście są oszustwami. Jednakże, czułość dla klasy 1 wynosi 0.92, co oznacza, że istnieje niewielkie ryzyko pominięcia niektórych przypadków oszustw.

#Podsumowując, na podstawie analizy wyników oceny, można stwierdzić, że model Random Forest jest skuteczny w klasyfikacji obserwacji klasy 0, ale może być nieco mniej skuteczny w wykrywaniu wszystkich przypadków oszustw (klasa 1). Jednakże, jego wysoka precyzja sugeruje, że większość przypadków sklasyfikowanych jako oszustwa jest rzeczywiście oszustwami. Dlatego też, w kontekście detekcji oszustw, model Random Forest wydaje się być dobrym wyborem.